In [12]:
import numpy as np

list_j = ['data', 'ai', 'data', 'data science', 'data scraping', 'data', 'database', 'data scraping', 'software', 'data analytics', 'python', 'database querying', 'mysql']
list_r = ['python', 'sql', 'data visualization', 'data', 'python', 'r', 'data science', 'machine']

list_r =  ['ma', 's', 'data visualization', 'analytics', 'databricks', 'data management', 'git', 'machine learn', 'matlab', 'java', 'scikit', 'python', 'gcp', 'r', 'sql', 'javascript', 'dart', 'sas', 'or', 'ge', 'data science', 'database management', 'n', 'p', 'neural', 'machine', 'pyspark', 'sea', 'excel', 'vision']
list_j =  ['python', 'data', 'database', 'ai', 'data scraping', 'software', 'data analytics', 'database querying', 'data science']

list_j = ['data scraping', 'ai', 'data science', 'database', 'data', 'data analytics', 'software', 'database querying', 'python']
list_r = ['dart', 'sas', 'ge', 'analytics', 'neural', 'excel', 'sea', 'ma', 'or', 'data management', 'n', 'scikit', 'sql', 'git', 'database management', 'machine', 's', 'data visualization', 'machine learn', 'vision', 'r', 'gcp', 'data science', 'javascript', 'matlab', 'pyspark', 'java', 'databricks', 'p', 'python']



In [13]:
def make_embeddings(client, chunks):
    '''
    This function creates embeddings for the chunks of text using the OpenAI API.
    '''
    
    def _make_embedding(client, chunk, model="text-embedding-3-small"):
        chunk = chunk.replace("\n", " ")
        return client.embeddings.create(input = [chunk], model=model).data[0].embedding
    
    embeddings = []
    for chunk in chunks:
        embedding = _make_embedding(client, chunk)
        embeddings.append(embedding)
    return embeddings

def make_meta_embedding(embeddings, skills):
    meta_embedding = {}
    for i, embedding in enumerate(embeddings):
        meta_embedding[skills[i]] = embedding
    return meta_embedding

In [14]:
from dotenv import load_dotenv
import numpy as np
from openai import OpenAI
import os
from pinecone import Pinecone, ServerlessSpec
from tqdm import tqdm
from bs4 import BeautifulSoup
import requests

In [15]:
load_dotenv(override=True)
    
openai_key = os.getenv("OPENAI_KEY")

client = OpenAI(api_key=openai_key)

In [16]:
from sklearn.metrics.pairwise import cosine_similarity

In [17]:
meta_embedding_j = make_meta_embedding(make_embeddings(client, list_j), list_j)
meta_embedding_r = make_meta_embedding(make_embeddings(client, list_r), list_r)

meta_embedding_j_copy = meta_embedding_j.copy()
# Compute the cosine similarity between the meta embeddings of the two lists
# missing_skills = []


# Assuming meta_embedding_j and meta_embedding_r are dictionaries with skills as keys and embeddings as values
# similarities = {}

for skill_j, embedding_j in meta_embedding_j.items():
    for skill_r, embedding_r in meta_embedding_r.items():
        # Compute the cosine similarity between the two embeddings
        similarity = cosine_similarity([embedding_j], [embedding_r])[0][0]
        # print(f"Similarity between {skill_j} and {skill_r}: {similarity}")
        # if the similarity score is above 0.7, remove skills in the dictionary mata_embedding_j
        if similarity > 0.7 and skill_j in meta_embedding_j_copy.keys():
            # print(f"Similarity between {skill_j} and {skill_r}: {similarity}")
            # missing_skills.append(skill_j)
            del meta_embedding_j_copy[skill_j]

# Now similarities is a dictionary with skill pairs as keys and their cosine similarity as values

In [18]:
a = list(meta_embedding_j_copy.keys())
a

['data scraping', 'ai', 'data', 'software', 'database querying']

In [19]:
', '.join(a)

'data scraping, ai, data, software, database querying'